In [ ]:
!pip install transformers -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.1 MB/s eta 0:00:00


In [ ]:
!pip install "transformers[torch]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/alldata_1_for_kaggle.csv', encoding = "latin-1")
df.head(3)

,Unnamed: 0,0,a
0,0,Thyroid_Cancer,Thyroid surgery in children in a single insti...
1,1,Thyroid_Cancer,""" The adopted strategy was the same as that us..."
2,2,Thyroid_Cancer,coronary arterybypass grafting thrombosis ï¬b...


In [ ]:
df.dropna(inplace=True)

In [ ]:
df = df.drop('Unnamed: 0', axis = 1)

In [ ]:
df = df.rename({'0': 'target', 'a': 'feature'}, axis = 1)

In [ ]:
label_mapping = {'Thyroid_Cancer' : 0, 'Colon_Cancer': 1, 'Lung_Cancer': 2}
df['target'] = df['target'].replace(label_mapping)
df.head(3)

,target,feature
0,0,Thyroid surgery in children in a single insti...
1,0,""" The adopted strategy was the same as that us..."
2,0,coronary arterybypass grafting thrombosis ï¬b...


In [ ]:
"""
Now we do Data Preprocessing.
"""
import string, re, nltk
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')


regexp = RegexpTokenizer("[\w']+")

def convert_to_lowercase(text):
    return text.lower()
def remove_whitespace(text):
    return text.strip()
def remove_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "")
    return text.translate(str.maketrans("", "", punct_str))
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)
def remove_http(text):
    http = "https?://\S+|www\.\S+"
    pattern = r"({})".format(http)
    return re.sub(pattern, "", text)
# Stopwords
stops = stopwords.words("english")
addstops = ["among", "onto", "shall", "thrice", "thus", "twice", "unto", "us", "would"]
allstops = stops + addstops
def remove_stopwords(text):
    return " ".join([word for word in regexp.tokenize(text) if word not in allstops])
stemmer = PorterStemmer()
def text_stemmer(text):
    text_stem = " ".join([stemmer.stem(word) for word in regexp.tokenize(text)])
    return text_stem
def discard_non_alpha(text):
    word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
    text_non_alpha = " ".join(word_list_non_alpha)
    return text_non_alpha

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def text_normalizer(text):
    text = convert_to_lowercase(text)
    text = remove_whitespace(text)
    text = re.sub('\n' , '', text)
    text = re.sub('\[.*?\]', '', text)
    text = remove_http(text)
    text = remove_punctuation(text)
    text = remove_html(text)
    text = remove_stopwords(text)
    text = discard_non_alpha(text)
    return text

In [ ]:
df['feature'] = df['feature'].apply(text_normalizer)

In [ ]:
df.shape

(7570, 2)

In [ ]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.0/226.0 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 11.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
import optuna

In [ ]:

seed = 2023
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
X = list(df['feature'])
y = list(df['target'])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length = 512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length = 512)

In [ ]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [ ]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average= 'weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# Define a function to optimize
def objective(trial):
    model_name = 'bert-base-uncased'
    num_labels = 3

    # Define hyperparameters to search
    num_train_epochs = trial.suggest_int('num_train_epochs', 1, 3)
    per_device_train_batch_size = trial.suggest_int('per_device_train_batch_size', 8, 16)
    learning_rate  = trial.suggest_float('learning_rate', 1e-6, 1e-4, log = True)

    # Define the Trainer
    args = TrainingArguments(
        output_dir="output",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        learning_rate = learning_rate,
    )

    trainer = Trainer(
        model=BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels),
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    # Train and evaluate the model
    trainer.train()
    results = trainer.evaluate()

    return results['eval_accuracy']


In [ ]:
# Create a study for optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)  # You can increase n_trials for more iterations

[I 2023-10-20 18:52:28,743] A new study created in memory with name: no-name-ca2ae8f8-4df2-4d4a-9922-c9472e1efde6
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.803300
1000,0.525700


[I 2023-10-20 19:23:18,882] Trial 0 finished with value: 0.9009247027741083 and parameters: {'num_train_epochs': 3, 'per_device_train_batch_size': 13, 'learning_rate': 2.3292722267855457e-06}. Best is trial 0 with value: 0.9009247027741083.


<class 'transformers.trainer_utils.EvalPrediction'>


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

<class 'transformers.trainer_utils.EvalPrediction'>


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
500,0.931800
1000,0.715000


[I 2023-10-20 21:09:36,570] Trial 4 finished with value: 0.6624834874504624 and parameters: {'num_train_epochs': 3, 'per_device_train_batch_size': 16, 'learning_rate': 1.0831271486375963e-06}. Best is trial 3 with value: 0.9907529722589168.


<class 'transformers.trainer_utils.EvalPrediction'>


In [ ]:
# Print the best hyperparameters
print("Best trial:")
best_trial = study.best_trial
print("Value: ", best_trial.value)
print("Params: ")
for key, value in best_trial.params.items():
    print(f"  {key}: {value}")

Best trial:
Value:  0.9907529722589168
Params: 
  num_train_epochs: 3
  per_device_train_batch_size: 11
  learning_rate: 8.448999142099342e-06
